In [1]:
import os
import json
import torch
import numpy as np
from tqdm import tqdm
from collections import Counter

In [2]:
#Load the cui data
# os.chdir('P:\ORD_Singh_201911038D\Justin\Stroke_Notes_13OCT21') #make sure we're in the right place
DATA_PATH = "P:\ORD_Singh_201911038D\Justin\Stroke_Notes_13OCT21"
with open(os.path.join(DATA_PATH, "sids_to_clamp_cuis.json"),'r') as infile:
    cuidata = json.load(infile)

In [3]:
#print(list(cuidata)[0]) # print the first document ID
print(cuidata[list(cuidata)[0]].keys()) # Just to show the data fields in each document
print(cuidata[list(cuidata)[0]]['cuis'][:10]) # show the first 10 cuis of this specific document

dict_keys(['cuis', 'semtype', 'presence'])
['C0008031', 'C1507320', 'C0015031', 'C4718442', 'C2707412', 'C0281822', 'C0043250', 'C0281822', 'C0398266', 'C3244243']


In [4]:
# We will get training pairs. One is the context term the next is the target term.
trainingpairs = []
vocab = dict()
frequency = Counter()
idx = 0

# This could absolutely be accomplished in an alternative / faster way but it's sufficiently fast for now
for doc in tqdm(cuidata):
    cuis = cuidata[doc]['cuis']
    cuiids = []
    for x in cuis:
        if x not in vocab.keys():
            vocab.update({x:idx})
            idx+=1
        cuiids += [vocab[x]]
    for i in range(len(cuiids)-1):
        pair = cuiids[i:i+2]
        if len(pair) < 2:
            continue
        if pair[0] != pair[1]:
            trainingpairs += [pair] #no self references
            frequency.update(pair)
print(trainingpairs[:3])
print(len(trainingpairs))
print(len(vocab))
assert np.all(np.asarray([x for x in vocab.values()]) == np.arange(len(vocab)))
# We'll use an embedding dimension of 50 to start
EMBEDDING_DIM = 100

100%|███████████████████████████████████████████████████████████████████████████| 29865/29865 [00:34<00:00, 866.05it/s]

[[0, 1], [1, 2], [2, 3]]
3526722
111303


In [5]:
frequency = np.asarray([values for key,values in frequency.items()])**0.75
frequency /= np.linalg.norm(frequency, ord=1)
frequency = torch.from_numpy(frequency)
samplingids = torch.arange(0, len(frequency))

In [6]:
class CUIEmbeddingModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CUIEmbeddingModel, self).__init__()
        self.dim = embedding_dim
        self.in_embeddings = torch.nn.Embedding(vocab_size, embedding_dim, sparse=True)
        self.out_embeddings = torch.nn.Embedding(vocab_size, embedding_dim, sparse=True)
        
    def init(self):
        self.in_embeddings.weight.data.uniform_(-0.5/self.dim, 0.5/self.dim) # scaled by dimensionality to control initial norm.
        self.out_embeddings.weight.data.uniform(-0, 0) # all 0s
        
    def forward(self, inputs, targets, negatives):
        inembed = self.in_embeddings(inputs)
        outembed = self.out_embeddings(targets)
        pos_score = torch.sum(torch.mul(inembed, outembed), dim=1)
        pos_score = torch.nn.functional.logsigmoid(pos_score)
        negembed = self.out_embeddings(negatives)
        neg_score = torch.bmm(negembed, inembed.unsqueeze(2)).squeeze()
        neg_score = torch.nn.functional.logsigmoid(-1*neg_score)
        return -1 * (torch.sum(pos_score)+torch.sum(neg_score))
    

In [7]:
losses = []
model = CUIEmbeddingModel(len(vocab), EMBEDDING_DIM)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
NEGSAMPLES = 2

In [8]:
#Test batching (also gives you an idea of theoretical max throughput in a way)
BATCH_SIZE=2000
for batchidx in tqdm(np.arange(0, len(trainingpairs), BATCH_SIZE), total = len(trainingpairs)//BATCH_SIZE):
    data = trainingpairs[batchidx:batchidx+BATCH_SIZE]
    context_idxs = torch.tensor(data)

1764it [00:01, 930.75it/s]                                                                                             


In [9]:
def nearest_embedding_search(cui):
    #C0948008 = ischemic stroke
    for i in zip(*torch.topk(torch.nn.functional.cosine_similarity(model.in_embeddings.weight.data[vocab[cui]].view(1,-1), 
                                                                   model.in_embeddings.weight.data), 10, largest=True)):
        print(f'{i[0]:0.3f}\t{cuitranslate[id2vocab[int(i[1])]]}')

In [10]:
id2vocab = {value:key for key,value in vocab.items()}
with open(os.path.join(DATA_PATH, "cuitranslate.json"),'r') as infile:
    cuitranslate = json.load(infile)

In [11]:
print(list(id2vocab.items())[0:10])
print(list(cuitranslate.items())[0:10])
print(cuitranslate["C0000005"])

[(0, 'C0008031'), (1, 'C1507320'), (2, 'C0015031'), (3, 'C4718442'), (4, 'C2707412'), (5, 'C0281822'), (6, 'C0043250'), (7, 'C0398266'), (8, 'C3244243'), (9, 'C0018802')]
[('C0000005', '(131)I-Macroaggregated Albumin'), ('C0000039', '1,2-dipalmitoylphosphatidylcholine'), ('C0000052', '1,4-alpha-Glucan Branching Enzyme'), ('C0000074', '1-Alkyl-2-Acylphosphatidates'), ('C0000084', '1-Carboxyglutamic Acid'), ('C0000096', '1-Methyl-3-isobutylxanthine'), ('C0000097', '1-Methyl-4-phenyl-1,2,3,6-tetrahydropyridine'), ('C0000098', '1-Methyl-4-phenylpyridinium'), ('C0000102', '1-Naphthylamine'), ('C0000103', '1-Naphthylisothiocyanate')]
(131)I-Macroaggregated Albumin


In [12]:
trainingpairs = torch.LongTensor(trainingpairs)
for epoch in range(50):
    total_loss = 0
    shuffler = torch.randperm(trainingpairs.shape[0])
    trainingpairs = trainingpairs[shuffler].view(trainingpairs.size())
    with tqdm(np.arange(0, len(trainingpairs), BATCH_SIZE), desc=f'Epoch {epoch+1}', total = len(trainingpairs)//BATCH_SIZE) as progress: #goes one example at a time
        for batchidx in progress:
            data = trainingpairs[batchidx:batchidx+BATCH_SIZE]
            inputs = torch.cat((data[:, 0], data[:,1])) #we'll go bidirectional; usually not done I suppose
            targets = torch.cat((data[:, 1], data[:,0]))#doubles the batch size
            negatives = samplingids[frequency.multinomial(num_samples=inputs.shape[0]*NEGSAMPLES, replacement=True)].reshape(inputs.shape[0], NEGSAMPLES)
            optimizer.zero_grad()
            loss = model.forward(inputs, targets, negatives)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            progress.set_postfix(loss=f"{loss.item():3.3f}")
        losses.append(total_loss)
        progress.set_postfix(loss=f"{total_loss:3.3f}")
    nearest_embedding_search('C0948008')
    
print(losses)


Epoch 1: 1764it [07:58,  3.68it/s, loss=9492.232]                                                                      


1.000	Ischemic stroke
0.415	Pathological fracture of left rib
0.386	spindle cell rhabdomyosarcoma of soft tissue
0.375	Trimalleolar Fractures
0.373	Self-expanding stent
0.372	Bundle of His
0.369	magnetic resonance imaging of knee: non-displaced fracture
0.369	pomalidomide
0.364	Cisternal tap
0.357	Color Blindness, Blue


Epoch 2: 1764it [08:10,  3.60it/s, loss=7536.564]                                                                      


1.000	Ischemic stroke
0.391	Pathological fracture of left rib
0.385	Trimalleolar Fractures
0.384	pomalidomide
0.376	spindle cell rhabdomyosarcoma of soft tissue
0.374	Femoracetabular Impingement
0.368	Guidance for cryoablation:Find:Pt:Chest>Lung:Doc:{Imaging modality}
0.365	Pertinent information
0.360	CAT scan of head
0.359	stress test-induced anxiety


Epoch 3: 1764it [08:04,  3.64it/s, loss=6796.781]                                                                      


...

1.000	Ischemic stroke
0.940	Acute Cerebrovascular Accidents
0.922	Acute ischemic stroke subtype
0.913	Cerebellar stroke
0.901	Thalamic infarction
0.892	transient ischemic attack without residual deficits
0.889	Embolic Stroke
0.887	Stroke, Lacunar
0.887	Transient Ischemic Attack
0.886	WEAKNESS RESIDUAL


Epoch 49: 1764it [12:21,  2.38it/s, loss=2524.348]                                                                     


1.000	Ischemic stroke
0.941	Acute Cerebrovascular Accidents
0.914	Acute ischemic stroke subtype
0.911	Cerebellar stroke
0.900	Thalamic infarction
0.894	transient ischemic attack without residual deficits
0.892	CANCER WORKUP
0.890	Embolic Stroke
0.886	Right sided cerebral hemisphere cerebrovascular accident
0.886	Stroke, Lacunar


Epoch 50: 1764it [11:14,  2.62it/s, loss=2744.672]                                                                     


1.000	Ischemic stroke
0.942	Acute Cerebrovascular Accidents
0.920	Acute ischemic stroke subtype
0.905	Cerebellar stroke
0.900	transient ischemic attack without residual deficits
0.899	Thalamic infarction
0.889	Stroke, Lacunar
0.888	WEAKNESS RESIDUAL
0.888	Embolic Stroke
0.883	Transient Ischemic Attack
[57257052.21484375, 40678626.90625, 35044751.833496094, 31836171.319335938, 29633419.258789062, 27983979.967773438, 26664647.645507812, 25576521.196289062, 24623284.225097656, 23813744.600097656, 23080688.614257812, 22425428.349609375, 21829675.135742188, 21282920.526367188, 20788439.360351562, 20319894.665039062, 19887428.76513672, 19492024.178955078, 19117606.393310547, 18766285.12109375, 18439591.645751953, 18119113.669433594, 17833568.506347656, 17558088.908691406, 17286365.635253906, 17047605.104736328, 16808459.82080078, 16573014.188964844, 16348047.401855469, 16142642.270996094, 15942748.134033203, 15753041.085449219, 15573979.71484375, 15383281.936279297, 15210348.486083984, 15052

In [14]:
for _, i in enumerate(losses):
    if losses[_-1] < i: #if loss is always monitonically decreasing, should print 0 only
        print(_)

In [15]:
# ToDo: Build document vectors from CUI vector components
docvectors = np.zeros((len(cuidata), EMBEDDING_DIM))
for i,doc in tqdm(enumerate(cuidata), total=len(cuidata)):
    for cui in cuidata[doc]['cuis']:
        docvectors[i] += model.in_embeddings.weight.data[vocab[cui]].numpy()


100%|███████████████████████████████████| 29865/29865 [01:16<00:00, 391.28it/s]


In [21]:
# ToDo: Build document vectors from CUI vector components
docvectors = np.zeros((len(cuidata), EMBEDDING_DIM))
for i,doc in tqdm(enumerate(cuidata), total=len(cuidata)):
    tmpvecs = np.zeros(EMBEDDING_DIM)
    for cui in cuidata[doc]['cuis']:
        tmpvecs += model.in_embeddings.weight.data[vocab[cui]].numpy()
    docvectors[i] = tmpvecs / len(cuidata[doc]['cuis'])

100%|███████████████████████████████████| 29865/29865 [01:07<00:00, 439.69it/s]


In [22]:
with open('average_document_vectors_28_FEB_2022.json','w') as outfile:
    json.dump({x:list(y) for x,y in zip(cuidata.keys(), docvectors)}, outfile)

In [16]:
with open("cui_vectors_15_NOV_2021.json",'w') as outfile:
    json.dump({key:model.in_embeddings.weight.data[value].tolist() for key,value in vocab.items()}, outfile)

In [25]:
torch.save(model.state_dict(), 'cui_model_15_NOV_2021.pt')

In [27]:
'''
To load the saved pytorch model:

model = CUIEmbeddingModel(len(vocab), EMBEDDING_DIM) #vocab and embedding dim must match what the model was trained on
model.load_state_dict(torch.load(cui_model_15_NOV_2021.pt))
model.eval() #set to evaluation mode
''';

In [32]:
nearest_embedding_search('C0742946')

1.000	CVA ETIOLOGY HEMORRHAGIC ISCHEMIC
0.847	Acute Cerebrovascular Accidents
0.836	Ischemic stroke
0.829	Left hemiparesis
0.829	Structure of middle cerebral artery
0.822	Thalamic infarction
0.821	Transient Ischemic Attack
0.820	Evaluation
0.815	Acute ischemic stroke subtype
0.815	transient ischemic attack without residual deficits


In [33]:
import xgboost